In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
import hook_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [2]:
model: HookedTransformer = HookedTransformer.from_pretrained("gpt2-large")

data = load_dataset("stas/openwebtext-10k", split="train")
data = data['text']

with open('data/gpt2_large_spectrum_pre_bins.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

lower_bounds = torch.tensor(bin_edges[:-1]).contiguous().cuda()

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


Repo card metadata block was not found. Setting CardData to empty.


/opt/conda/lib/python3.10/site-packages/datasets/download/download_manager.py:527: FutureWarning:

'num_proc' was deprecated in version 2.6.2 and will be removed in 3.0.0. Pass `DownloadConfig(num_proc=<num_proc>)` to the initializer instead.



Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
layer, neuron = 31, 892

hook_name = f'blocks.{layer}.mlp.hook_pre'
hooks = [(hook_name, hook_utils.save_activation)]
max_range = np.array([i for i in range(model.cfg.n_ctx)])
an_tokens = torch.tensor([model.to_single_token(' an'), model.to_single_token(' AN'), model.to_single_token(' An')]).cuda()

prompt_indices = []
token_indices = []
tokens = []
next_tokens = []
next_is_ans = []
acts = []
bins = []
bin_counts = torch.zeros(lower_bounds.shape[0] + 1).cuda()

for i, prompt in tqdm(enumerate(data)):
    if len(prompt) < 20:
        continue
    all_tokens = model.to_tokens(prompt, truncate=True)[0]
    if all_tokens.shape[0] < 2:
        continue
    
    # Don't include the final token because we need the next token
    current_tokens = all_tokens[:-1]

    with model.hooks(hooks):
        model(current_tokens)

    current_acts = model.hook_dict[hook_name].ctx['activation'][0, :, neuron].contiguous()

    prompt_indices.append(np.array([i]*current_tokens.shape[0]))
    token_indices.append(max_range[:current_tokens.shape[0]])
    tokens.append(current_tokens.cpu().numpy())
    next_tokens.append(all_tokens[1:].cpu().numpy())
    acts.append(current_acts.cpu().numpy())
    next_is_ans.append((torch.isin(all_tokens[1:], an_tokens)).cpu().numpy())

    bin_indices = torch.searchsorted(lower_bounds, current_acts)
    bins.append(bin_indices.cpu().numpy())
    bin_counts += torch.bincount(bin_indices, minlength=bin_counts.shape[0])
    if i % 100 == 0:
        if (bin_counts > 20).all():
            break

0it [00:00, ?it/s]

In [7]:
prompt_indices = np.concatenate(prompt_indices)
token_indices = np.concatenate(token_indices)
tokens = np.concatenate(tokens)
next_tokens = np.concatenate(next_tokens)
next_is_ans = np.concatenate(next_is_ans)
acts = np.concatenate(acts)

ValueError: zero-dimensional arrays cannot be concatenated

In [5]:
df = pd.DataFrame({
    'prompt_index': prompt_indices,
    'token_index': token_indices,
    'token': tokens,
    'next_token': next_tokens,
    'next_is_an': next_is_ans,
    'activation': acts,
})

In [19]:
# Sort the DataFrame
df.sort_values('activation', inplace=True)
trimmed_df = df.iloc[5:-185]
df_shuffled = trimmed_df.sample(frac=1, random_state=0).reset_index(drop=True)

bin_labels = np.linspace(df_shuffled['activation'].min(), df_shuffled['activation'].max(), 21)
df_shuffled['L31N892 Activation'] = pd.cut(df_shuffled['activation'], bins=bin_labels)

# df_shuffled['L31N892 Activation'], bin_edges = pd.cut(df_shuffled['activation'], bins=20, labels=False, retbins=True)

sample_df = df_shuffled.groupby('L31N892 Activation').apply(lambda x: x.sample(n=min(len(x), 50), random_state=0)).reset_index(drop=True)

# Count items in each bin
bin_counts = sample_df['L31N892 Activation'].value_counts().sort_index()

print(bin_counts)

/tmp/ipykernel_9199/4095375515.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



L31N892 Activation
(-9.081, -8.171]    50
(-8.171, -7.262]    50
(-7.262, -6.352]    50
(-6.352, -5.443]    50
(-5.443, -4.533]    50
(-4.533, -3.624]    50
(-3.624, -2.714]    50
(-2.714, -1.805]    50
(-1.805, -0.895]    50
(-0.895, 0.0145]    50
(0.0145, 0.924]     50
(0.924, 1.834]      50
(1.834, 2.743]      50
(2.743, 3.653]      50
(3.653, 4.562]      50
(4.562, 5.472]      50
(5.472, 6.381]      50
(6.381, 7.291]      50
(7.291, 8.2]        50
(8.2, 9.11]         50
Name: count, dtype: int64


In [20]:
df_shuffled = df.sample(frac=1, random_state=0).reset_index(drop=True)
df_shuffled['L31N892 Activation'], bin_edges = pd.cut(df_shuffled['activation'], bins=20, labels=False, retbins=True)

sample_df = df_shuffled.groupby('L31N892 Activation').apply(lambda x: x.sample(n=min(len(x), 100), random_state=0)).reset_index(drop=True)
print(sample_df.head())
print(bin_edges)

with open('data/gpt2_large_spectrum_pre_v2.pkl', 'wb') as f:
    pickle.dump(sample_df, f)

with open('data/gpt2_large_spectrum_pre_bins_v2.pkl', 'wb') as f:
    pickle.dump(bin_edges, f)

   prompt_index  token_index  token  next_token  next_is_an  activation  \
0          6706          352    447         251       False   -9.637022   
1          2310           54    137          94       False   -9.080560   
2          6655         1007  26009          16       False   -8.583082   
3          6844          883  27371        2598       False   -8.629142   
4          8338          882    447         248       False   -8.436122   

   L31N892 Activation  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
[-9.66145449 -8.41539865 -7.19377527 -5.9721519  -4.75052853 -3.52890515
 -2.30728178 -1.08565841  0.13596497  1.35758834  2.57921171  3.80083508
  5.02245846  6.24408183  7.4657052   8.68732858  9.90895195 11.13057532
 12.3521987  13.57382207 14.79544544]


In [10]:
with open('data/gpt2_large_spectrum_pre_v2.pkl', 'rb') as f:
    sample_df = pickle.load(f)

with open('data/gpt2_large_spectrum_pre_bins_v2.pkl', 'rb') as f:
    bin_edges = pickle.load(f)

In [17]:
grouped = sample_df.groupby('L31N892 Activation', as_index=False)['next_is_an'].agg(['sum', 'count'])
grouped['Next token is \' an\''] = grouped['sum'] / grouped['count']
grouped['Next token is not \' an\''] = 1 - grouped['Next token is \' an\'']

grouped_melted = grouped.melt(id_vars=['L31N892 Activation'], value_vars=['Next token is \' an\'', 'Next token is not \' an\''], 
                            var_name='status', value_name='Proportion of Labels')
fig = px.area(grouped_melted, x='L31N892 Activation', y='Proportion of Labels', color='status', line_shape='spline')
tickvals = np.arange(len(bin_edges) - 1)
ticktext = [f"{bin_edges[i]:.2f}-{bin_edges[i+1]:.2f}" for i in tickvals]

fig.update_layout(
    yaxis=dict(range=[0, 1]),
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
)
fig.show()

In [42]:
merged[['L31N892 Activation', 'Proportion of Labels']].groupby('L31N892 Activation').sum()


,Proportion of Labels
L31N892 Activation,
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0


In [56]:
# Group by 'bucket' and 'next_token', then calculate proportions
# Separates into groups by bucket and next_token, adds size columns, then converts back to dataframe via reset index
grouped_by_bucket_and_next_token = sample_df.groupby(['L31N892 Activation', 'next_token']).size().reset_index(name='count')
total = grouped_by_bucket_and_next_token.groupby('L31N892 Activation')['count'].sum().reset_index(name='total_count')
merged = pd.merge(grouped_by_bucket_and_next_token, total, on='L31N892 Activation')
merged['activation'] = sample_df['activation']
merged['Proportion of Labels'] = merged['count'] / merged['total_count']

# Sort data by commonality of next token
total_counts = merged.groupby('next_token')['Proportion of Labels'].sum().reset_index()
sorted_tokens = total_counts.sort_values('Proportion of Labels', ascending=False)['next_token'].tolist()

merged['next_token'] = pd.Categorical(merged['next_token'], categories=sorted_tokens, ordered=True)
merged.sort_values(['next_token', 'L31N892 Activation'], inplace=True)
merged['Next Token'] = merged['next_token'].apply(lambda x: model.to_single_str_token(x) + '                   ' + str(np.random.random()))

# Create plot
fig = px.area(merged, x='L31N892 Activation', y='Proportion of Labels', color='Next Token', line_shape='spline')
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ),
    yaxis=dict(range=[0, 1]))
fig.show()

# Merge in other forms of an?
# Second chart: scale each bucket by proportion of dataset that falls into that bucket, e.g. proportion * 0.01. Should result in a bell curve
# Third chart: Activation value * Pr(neuron being in bucket)

In [60]:
merged

,L31N892 Activation,next_token,count,total_count,activation,Proportion of Labels,Next Token,Scaled Proportion of Labels
429,7,281,1,100,-2.816659,0.01,an 0.1520248531286481,0.002161
564,9,281,10,100,-1.796834,0.10,an 0.1520248531286481,0.000239
621,10,281,18,100,-1.250345,0.18,an 0.1520248531286481,0.000077
676,11,281,26,100,-0.716163,0.26,an 0.1520248531286481,0.000030
720,12,281,36,100,-0.712553,0.36,an 0.1520248531286481,0.000017
...,...,...,...,...,...,...,...,...
529,8,1969,1,100,-1.606101,0.01,close 0.6006510665483559,0.000226
97,2,1962,1,100,-7.507118,0.01,Gu 0.7435939161765598,0.000035
148,3,1910,1,100,-6.000294,0.01,market 0.5062660594952028,0.000201
656,10,1903,1,100,0.033512,0.01,early 0.6341040313763313,0.000004


In [65]:
def get_scaled_proportion(row):
    bucket_size = bucket_sizes[bucket_sizes['L31N892 Activation'] == row['L31N892 Activation']].iloc[0, -1].item()
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

bucket_sizes = df_shuffled.groupby('L31N892 Activation').size().reset_index()
total = bucket_sizes.iloc[:, -1].sum()


def sdf(x):
    # print(x)
    return x[['next_token']] == model.to_single_token(' an')

merged['Scaled Proportion of Labels'] = merged.apply(get_scaled_proportion, axis=1)
merged['Next Token is \' an\''] = merged.apply(sdf, axis=1)

In [66]:
merged

,L31N892 Activation,next_token,count,total_count,activation,Proportion of Labels,Next Token,Scaled Proportion of Labels,Next Token is ' an'
429,7,281,1,100,-2.816659,0.01,an 0.1520248531286481,0.002161,True
564,9,281,10,100,-1.796834,0.10,an 0.1520248531286481,0.000239,True
621,10,281,18,100,-1.250345,0.18,an 0.1520248531286481,0.000077,True
676,11,281,26,100,-0.716163,0.26,an 0.1520248531286481,0.000030,True
720,12,281,36,100,-0.712553,0.36,an 0.1520248531286481,0.000017,True
...,...,...,...,...,...,...,...,...,...
529,8,1969,1,100,-1.606101,0.01,close 0.6006510665483559,0.000226,False
97,2,1962,1,100,-7.507118,0.01,Gu 0.7435939161765598,0.000035,False
148,3,1910,1,100,-6.000294,0.01,market 0.5062660594952028,0.000201,False
656,10,1903,1,100,0.033512,0.01,early 0.6341040313763313,0.000004,False


In [68]:
merged[['L31N892 Activation', 'Scaled Proportion of Labels', 'Next Token is \' an\'']]

,L31N892 Activation,Scaled Proportion of Labels,Next Token is ' an'
429,7,0.002161,True
564,9,0.000239,True
621,10,0.000077,True
676,11,0.000030,True
720,12,0.000017,True
...,...,...,...
529,8,0.000226,False
97,2,0.000035,False
148,3,0.000201,False
656,10,0.000004,False


In [70]:
fig = px.area(merged, x='L31N892 Activation', y='Scaled Proportion of Labels', line_shape='linear', title='Scaled proportion of labels')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()

In [15]:
merged

,L31N892 Activation,next_token,count,total_count,activation,Proportion of Labels,Next Token,Scaled Proportion of Labels
429,7,281,1,100,-2.816659,0.01,an,0.002161
564,9,281,10,100,-1.796834,0.10,an,0.000239
621,10,281,18,100,-1.250345,0.18,an,0.000077
676,11,281,26,100,-0.716163,0.26,an,0.000030
720,12,281,36,100,-0.712553,0.36,an,0.000017
...,...,...,...,...,...,...,...,...
529,8,1969,1,100,-1.606101,0.01,close,0.000226
97,2,1962,1,100,-7.507118,0.01,Gu,0.000035
148,3,1910,1,100,-6.000294,0.01,market,0.000201
656,10,1903,1,100,0.033512,0.01,early,0.000004


In [21]:
# Third chart: Activation value * Pr(neuron being in bucket)

def get_scaled_proportion(row):
    bucket_size = bucket_sizes[bucket_sizes['L31N892 Activation'] == row['L31N892 Activation']].iloc[0, -1].item()
    scaling_value = bucket_size / total
    return row['Proportion of Labels'] * scaling_value

bucket_sizes = df_shuffled.groupby('L31N892 Activation').size().reset_index()
total = bucket_sizes.iloc[:, -1].sum()

merged['Expected Value'] = merged['activation'] * merged['Scaled Proportion of Labels']


fig = px.area(merged, x='L31N892 Activation', y='Expected Value', color='Next Token', line_shape='spline')

fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=tickvals,
        ticktext=ticktext
    ))
fig.show()